## Murphy and Applegate
### Anomaly Detection Project
#### Tyler's First Notebook

In [1]:
from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

from env import host, user, password
import acquire
import prepare
import explore

In [2]:
df = acquire.get_cohort_curr_data()

In [3]:
df.head()

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2


In [4]:
df.shape

(847330, 15)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 847330 entries, 0 to 847329
Data columns (total 15 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   date        847330 non-null  object 
 1   time        847330 non-null  object 
 2   path        847329 non-null  object 
 3   user_id     847330 non-null  int64  
 4   cohort_id   847330 non-null  float64
 5   ip          847330 non-null  object 
 6   id          847330 non-null  int64  
 7   name        847330 non-null  object 
 8   slack       847330 non-null  object 
 9   start_date  847330 non-null  object 
 10  end_date    847330 non-null  object 
 11  created_at  847330 non-null  object 
 12  updated_at  847330 non-null  object 
 13  deleted_at  0 non-null       float64
 14  program_id  847330 non-null  int64  
dtypes: float64(2), int64(3), object(10)
memory usage: 103.4+ MB


### Initial Prep To-Do:
- Concat 'date' and 'time' to datetime and set as index
- Convert 'start_date', 'end_date', 'created_at', 'updated_at' to datetime
- Drop 'deleted_at'
- Create new column that maps out whether DS or WebDev

In [6]:
df = prepare.initial_prep(df)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 847329 entries, 2018-01-26 09:55:03 to 2021-04-21 16:44:39
Data columns (total 13 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   endpoint    847329 non-null  object        
 1   user_id     847329 non-null  int64         
 2   cohort_id   847329 non-null  float64       
 3   ip          847329 non-null  object        
 4   id          847329 non-null  int64         
 5   cohort      847329 non-null  object        
 6   slack       847329 non-null  object        
 7   start_date  847329 non-null  datetime64[ns]
 8   end_date    847329 non-null  datetime64[ns]
 9   created_at  847329 non-null  datetime64[ns]
 10  updated_at  847329 non-null  datetime64[ns]
 11  program_id  847329 non-null  int64         
 12  program     847329 non-null  object        
dtypes: datetime64[ns](4), float64(1), int64(3), object(5)
memory usage: 90.5+ MB


In [8]:
df.head()

,endpoint,user_id,cohort_id,ip,id,cohort,slack,start_date,end_date,created_at,updated_at,program_id,program
dt,,,,,,,,,,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,full_stack_php
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,full_stack_php
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,full_stack_php
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,1,full_stack_php
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,2,java


In [11]:
df.tail()

,endpoint,user_id,cohort_id,ip,id,cohort,slack,start_date,end_date,created_at,updated_at,program_id,program
dt,,,,,,,,,,,,,
2021-04-21 16:41:51,jquery/personal-site,64,28.0,71.150.217.33,28,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,java
2021-04-21 16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,28,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,java
2021-04-21 16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,28,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,java
2021-04-21 16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,28,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,java
2021-04-21 16:44:39,jquery/mapbox-api,64,28.0,71.150.217.33,28,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,2,java


2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

In [ ]:
# let's make a df for each program
df_1 = df[df['program_id'] ==1]
df_2 = df[df['program_id'] ==2]
df_3 = df[df['program_id'] ==3]
df_4 = df[df['program_id'] ==4]

In [ ]:
# Now let's make df w/o '/' endpoint
df_1a = df_1[df_1['endpoint'] != '/']
df_2a = df_2[df_2['endpoint'] != '/']
df_3a = df_3[df_3['endpoint'] != '/']
df_4a = df_4[df_4['endpoint'] != '/']

In [ ]:
# Now break them down so we only have info we need
df_1a = df_1a[['endpoint', 'cohort']]
df_2a = df_2a[['endpoint', 'cohort']]
df_3a = df_3a[['endpoint', 'cohort']]
df_4a = df_4a[['endpoint', 'cohort']]

In [ ]:
# Groupby 'endpoint' and get value counts for 'cohort'
df_1b = pd.DataFrame(df_1a.groupby('endpoint')['cohort'].value_counts()).rename(columns = {'cohort':'count'})
df_2b = pd.DataFrame(df_2a.groupby('endpoint')['cohort'].value_counts()).rename(columns = {'cohort':'count'})
df_3b = pd.DataFrame(df_3a.groupby('endpoint')['cohort'].value_counts()).rename(columns = {'cohort':'count'})
df_4b = pd.DataFrame(df_4a.groupby('endpoint')['cohort'].value_counts()).rename(columns = {'cohort':'count'})

In [ ]:
df_1b = df_1b.reset_index(level=['endpoint', 'cohort'])
df_2b = df_2b.reset_index(level=['endpoint', 'cohort'])
df_3b = df_3b.reset_index(level=['endpoint', 'cohort'])
df_4b = df_4b.reset_index(level=['endpoint', 'cohort'])

In [ ]:
df_1b.shape

### Program 1: Full-Stack

In [ ]:
df_1c = pd.DataFrame(df_1b.groupby(['endpoint'])['count'].agg(['min']))
df_1d = pd.DataFrame(df_1b.groupby(['endpoint'])['cohort'].agg(['min']))
df_1e = pd.DataFrame(df_1b.groupby(['endpoint'])['count'].agg(['max']))
df_1f = pd.DataFrame(df_1b.groupby(['endpoint'])['cohort'].agg(['max']))

In [ ]:
df_1g = pd.merge(df_1c, df_1d, how='left', on='endpoint')
df_1h = pd.merge(df_1e, df_1f, how='left', on='endpoint')
df_1i = pd.merge(df_1g, df_1h, how='left', on='endpoint')

In [ ]:
df_1i['range'] = df_1i['max_x']-df_1i['min_x']
df_1i[df_1i['min_x']>10].sort_values(by='range', ascending=False)

#### Key Findings & Takeaways:
- We only wanted to look at values where the min number of page views > 10
    - The assumption is that anything < 10 means it's probably not class related
- There are 2 lessons where the range is greater than 50 ('java-iii/jsp-and-jstl', 'java-iii/jsp-and-jstl') 
- Both show Olympic viewing the lessons far more often than Arches

### Program 2: Java

In [ ]:
df_2c = pd.DataFrame(df_2b.groupby(['endpoint'])['count'].agg(['min']))
df_2d = pd.DataFrame(df_2b.groupby(['endpoint'])['cohort'].agg(['min']))
df_2e = pd.DataFrame(df_2b.groupby(['endpoint'])['count'].agg(['max']))
df_2f = pd.DataFrame(df_2b.groupby(['endpoint'])['cohort'].agg(['max']))

In [ ]:
df_2g = pd.merge(df_2c, df_2d, how='left', on='endpoint')
df_2h = pd.merge(df_2e, df_2f, how='left', on='endpoint')
df_2i = pd.merge(df_2g, df_2h, how='left', on='endpoint')

In [ ]:
df_2i['range'] = df_2i['max_x']-df_2i['min_x']
df_2i[df_2i['min_x']>10].sort_values(by='range', ascending=False)

#### Key Findings & Takeaways:
- We only wanted to look at values where the min number of page views > 10
    - The assumption is that anything < 10 means it's probably not class related
- There is only 1 lesson where the range is greater than 50 ('html-css/css-ii/bootstrap-grid-system	')
- This shows Zion viewing the lesson far more often than Andromeda

### Program 3: Data Science

In [ ]:
df_3c = pd.DataFrame(df_3b.groupby(['endpoint'])['count'].agg(['min']))
df_3d = pd.DataFrame(df_3b.groupby(['endpoint'])['cohort'].agg(['min']))
df_3e = pd.DataFrame(df_3b.groupby(['endpoint'])['count'].agg(['max']))
df_3f = pd.DataFrame(df_3b.groupby(['endpoint'])['cohort'].agg(['max']))

In [ ]:
df_3g = pd.merge(df_3c, df_3d, how='left', on='endpoint')
df_3h = pd.merge(df_3e, df_3f, how='left', on='endpoint')
df_3i = pd.merge(df_3g, df_3h, how='left', on='endpoint')

In [ ]:
df_3i['range'] = df_3i['max_x']-df_3i['min_x']
df_3i[df_3i['min_x']>10].sort_values(by='range', ascending=False).head(20)

#### Key Findings & Takeaways:
- We only wanted to look at values where the min number of page views > 10
    - The assumption is that anything < 10 means it's probably not class related
- There countless lessons where various cohorts ('Florence, Easley, Darden') outviewed Bayes
- Some evidence to suggest positive relationship between cohort dates attended and page views

### Program 4: Front End

In [ ]:
df_4c = pd.DataFrame(df_4b.groupby(['endpoint'])['count'].agg(['min']))
df_4d = pd.DataFrame(df_4b.groupby(['endpoint'])['cohort'].agg(['min']))
df_4e = pd.DataFrame(df_4b.groupby(['endpoint'])['count'].agg(['max']))
df_4f = pd.DataFrame(df_4b.groupby(['endpoint'])['cohort'].agg(['max']))

In [ ]:
df_4g = pd.merge(df_4c, df_4d, how='left', on='endpoint')
df_4h = pd.merge(df_4e, df_4f, how='left', on='endpoint')
df_4i = pd.merge(df_4g, df_4h, how='left', on='endpoint')

In [ ]:
df_4i['range'] = df_4i['max_x']-df_4i['min_x']
df_4i[df_4i['min_x']>1].sort_values(by='range', ascending=False).head(20)

In [ ]:
df_4i

#### Key Findings & Takeaways:
- We only wanted to look at values where the min number of page views > 10
    - The assumption is that anything < 10 means it's probably not class related
- No values showed up (not much data for this program)
- Appears to only have been 1 cohort, for a very brief period of time

In [ ]:
import explore

6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?

#### Steps to answering question 6:
- [x] Reacquire data
- [x] Redo initial_prep
- [x] Use remove_home function from q2
- [x] Remove all enties where 'end_date' > index
- [x] Create q6_cols function that keeps 'index', 'cohort', 'endpoint', 'end_date', 'program_id'
- [x] Use split_by_program function from q2

In [ ]:
# Reacquire data
df = acquire.get_cohort_curr_data()

In [ ]:
# Redo initial_prep
df = prepare.initial_prep(df)

In [ ]:
# Use remove_home function
df = explore.remove_home(df)

In [ ]:
# Only return entries where student has graduated
df = explore.grads_only(df)

In [ ]:
# return only columns needed for q6
df = explore.q6_cols(df)

In [ ]:
# Use split_by_program function
df_1, df_2, df_3, df_4 = explore.split_by_program(df)

In [ ]:
df_1, df_2, df_3, df_4 = explore.endpoint_group(df_1, df_2, df_3, df_4)

In [ ]:
df_10 = explore.q6_math(df_1)

In [ ]:
df_10 = explore.add_zscore_cols(df_10)

In [ ]:
df_10[df_10['zscore']>3].sort_values(by='zscore', ascending=False)